In [ ]:
# ✅ ngrok バイナリをダウンロード＆展開＆配置
!wget -q -c https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/ngrok

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from huggingface_hub import login
from getpass import getpass

# 🔐 ngrok トークン
ngrok_token = getpass("🔐 ngrok token を入力してください: ")
!ngrok config add-authtoken {ngrok_token}

🔐 ngrok token を入力してください: ··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import subprocess
import time
import requests

# 🔄 ngrok 起動（バックグラウンド）
subprocess.Popen(["ngrok", "http", "5000"])

# ⏳ ngrokのURL取得まで待機
time.sleep(7)

try:
    res = requests.get("http://localhost:4040/api/tunnels")
    tunnels = res.json()["tunnels"]
    tunnel_url = [t["public_url"] for t in tunnels if t["proto"] == "https"][0]
    print("🌍 Colab構造化API URL（.envに貼る用）:")
    print(tunnel_url)
except Exception as e:
    print("❌ ngrok URL取得エラー:", e)




from flask import Flask, request, jsonify
import re, json, threading
import sys

app = Flask(__name__)

def extract_json(text):
    try:
        match = re.search(r"\{.*?\}", text, re.DOTALL)
        if match:
            json_str = match.group()
            print("🔍 抽出したJSON候補:", json_str)
            return json.loads(json_str)
        return {}
    except Exception as e:
        print("❌ JSON抽出エラー:", e)
        return {}

@app.route("/ping", methods=["GET"])
def ping():
    return "pong"

# analyze関数をこうする
@app.route("/analyze", methods=["POST"])
def analyze():
    with open("/content/flask_log.txt", "w") as f:
        f.write("🔥🔥🔥 /analyze が呼び出された！ 🔥🔥🔥\n")
        data = request.get_json()
        f.write(f"📥 受け取った自然文: {data}\n")
    return jsonify({
        "start_time": "2025-06-02T06:00:00",
        "end_time": "2025-06-02T07:00:00",
        "purpose": "研究でGPUを使うため",
        "priority_score": 0.90,
        "server_name": "A100",
        "received_text": data.get("text", "N/A")
    })

# ✅ Flask をバックグラウンドで起動（Colabが止まらない）
def run_flask():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_flask).start()


🌍 Colab構造化API URL（.envに貼る用）:
https://fcc6-34-90-194-233.ngrok-free.app
 * Serving Flask app '__main__'


In [ ]:
# ==============================
# 🔌 停止セル：ngrok と Flask を明示的に終了
# ==============================

import os
import signal

# ✅ ngrok のプロセス終了（プロセス名ベース）
!pkill ngrok

# ✅ Flask のスレッド（threading.Thread）を止めるには、通常はランタイム再起動が一番確実だが、
#     一応「サーバーを停止するルート」も用意しておけばリモート停止できる（ただしColabでは不安定）

import requests

try:
    # Flask に shutdown 要求を送る（アプリに /shutdown が定義されている場合）
    res = requests.post("http://localhost:5000/shutdown")
    print("🛑 Flask 停止応答:", res.text)
except Exception as e:
    print("⚠️ Flask 停止リクエスト失敗:", e)


INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:57:35] "POST /shutdown HTTP/1.1" 404 -


🛑 Flask 停止応答: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

